# Dataset

## Indo4B

In [ ]:
!wget https://storage.googleapis.com/babert-pretraining/IndoNLU_finals/dataset/preprocessed/dataset_wot_uncased_blanklines.tar.xz
!tar -xvf dataset_wot_uncased_blanklines.tar.xz --directory dataset

## CCNEWS-ID

In [ ]:
!wget https://storage.depia.wiki/ccnews-id.tar
!tar -xvf ccnews-id.tar --directory dataset

In [ ]:
!wget https://raw.githubusercontent.com/Wikidepia/indonesian_datasets/master/dictionary/wordlist/data/wordlist.txt

# Prepare the model

## Load tokenizer

In [1]:
import tensorflow as tf
from transformers import TFXLMRobertaModel, XLMRobertaTokenizer, TFXLMRobertaForMaskedLM, XLMRobertaConfig
import os
import numpy as np
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
from transformers import create_optimizer, AdamWeightDecay


model_name = 'distill'
config = XLMRobertaConfig.from_pretrained(model_name)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name, config=config)



You are using a model of type bert to instantiate a model of type xlm-roberta. This is not supported for all configurations of models and can yield errors.


## Create student model with teacher weights

In [ ]:
student_model = TFXLMRobertaForMaskedLM.from_pretrained('intfloat/multilingual-e5-small', from_pt=True, config=config)

student_model.summary(expand_nested=True)

## Load teacher model

In [2]:
teacher_model = TFXLMRobertaForMaskedLM.from_pretrained('intfloat/multilingual-e5-small', from_pt=True)

teacher_model.summary(expand_nested=True)

You are using a model of type bert to instantiate a model of type xlm-roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForMaskedLM: ['embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFXLMRobertaForMaskedLM were not initialized from the PyTorch model and are newly initialized: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_hea

Model: "tfxlm_roberta_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFXLMRobertaMainL  multiple                  117505920 
 ayer)                                                           
                                                                 
 lm_head (TFXLMRobertaLMHea  multiple                  96610997  
 d)                                                              
                                                                 
Total params: 117904565 (449.77 MB)
Trainable params: 117904565 (449.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Prepare distilled model

In [4]:
inputs = ???


student_embedding = student_model.roberta
student_embedding_out = student_embedding(inputs)
student_mlm_out = student_model(inputs)

model = tf.keras.Model(inputs, [student_embedding_out, student_mlm_out], name="distilled_xlmroberta")


loss={"head1": "mse", "head2": "mse"}


SyntaxError: invalid syntax (2829640424.py, line 1)

## Prepare training Datasets

In [3]:
PRETRAINING_BATCH_SIZE = 126
PRETRAINING_LEARNING_RATE = 5e-4
PRETRAINING_EPOCHS = 8
VOCABULARY_SIZE = 250002
MASK_RATE = 0.25
PREDICTIONS_PER_SEQ = 32
SEQ_LENGTH = 512


folder = 'dataset/processed_uncased_blanklines/'
file_list = os.listdir(folder)
file_list = [f"{folder}/{_file}" for _file in file_list]

datasets = load_dataset("text", data_files={"train": file_list[-1]})


Found cached dataset text (/Users/mdaniyalk/.cache/huggingface/datasets/text/default-484ac364d0d2d9cd/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [5]:
tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["text"]
)

Map (num_proc=4):   0%|          | 0/36866419 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (900 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1057 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (639 > 512). Running this sequence through the model will result in indexing errors


In [6]:
block_size = 64

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, though you could add padding instead if the model supports it
    # In this, as in all things, we advise you to follow your heart
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [7]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=8,
)

Map (num_proc=8):   0%|          | 0/36866419 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15, return_tensors="np"
)

In [9]:
train_set = teacher_model.prepare_tf_dataset(
    lm_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

In [ ]:
print('a')

In [10]:
lm_datasets['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7633402
})

In [11]:
lm_datasets['train']['input_ids']

[[0,
  106231,
  51511,
  86422,
  1566,
  25464,
  66,
  9272,
  351,
  199387,
  56,
  4407,
  82962,
  115025,
  15,
  978,
  4867,
  16,
  62886,
  6,
  10681,
  190,
  128191,
  73155,
  103629,
  160246,
  3510,
  5,
  2,
  0,
  2651,
  2215,
  7,
  60191,
  12477,
  247,
  311,
  100179,
  2755,
  9536,
  15,
  350,
  38081,
  16,
  104,
  301,
  79,
  59505,
  84812,
  482,
  106165,
  144261,
  26405,
  373,
  4867,
  8980,
  11179,
  673,
  282,
  1414,
  45,
  8731,
  166680,
  94488],
 [117,
  226968,
  66,
  15,
  26775,
  16,
  1059,
  5128,
  1463,
  482,
  3262,
  387,
  2651,
  704,
  68183,
  20741,
  2651,
  2215,
  7,
  5,
  2,
  0,
  2,
  0,
  8980,
  11179,
  4,
  119,
  12043,
  311,
  311,
  100179,
  2755,
  9536,
  49393,
  6370,
  2839,
  1166,
  704,
  3719,
  2848,
  24043,
  3643,
  734,
  154902,
  31856,
  49393,
  77790,
  4,
  63347,
  144932,
  462,
  8845,
  66150,
  39461,
  5,
  44,
  2,
  0,
  1216,
  144932,
  4,
  119019,
  4],
 [123,
  112786,


In [16]:
unbatched_dataset = train_set.unbatch()
for idx, example in enumerate(unbatched_dataset):
    print(example)
    if idx == 0:
        break

({'input_ids': <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([ 75347,  26881, 225557,    485,  68892,  14749, 250001,  55933,
       182079, 250001,   6202,     21,  92224,  35228,  12893,   9639,
          273,  10532,  55341,      5,      2,      0,     21,   2335,
        65638,  23995, 250001, 250001,  91843,      5,      2,      0,
         4956,   9884,  10460,  14391,    734, 140890,      5,      2,
            0,      2,      0,    738,   5619,  58248,    399,     21,
        34281,    102,   1384, 225557,     21,   2335,    119, 224364,
       125274,    120,  11727,  35435,     85,     13, 238241,      2])>, 'attention_mask': <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>}, <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([  -100,   -100,   -100,   -100,   -10

# Traning the models

## Fine-tune teacher models

In [ ]:
optimizer = AdamWeightDecay(learning_rate=1e-3, weight_decay_rate=0.01)

teacher_model.compile(optimizer=optimizer, jit_compile=True, metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)])

In [ ]:
model.roberta.trainable = False
teacher_model.fit(train_set, epochs=2)
model.roberta.trainable = True
teacher_model.fit(train_set, epochs=2)
model.roberta.trainable = False
teacher_model.fit(train_set, epochs=1)

In [ ]:
teacher_model.save_pretrained(save_directory='teacher_e5', saved_model=true)

In [19]:
inputs = tf.keras.layers.Input(shape=(len(lm_datasets['train']['input_ids'][0]),), dtype=tf.int32, name="input_ids")
embedding = teacher_model.roberta
outputs = embedding(inputs).last_hidden_state
model = tf.keras.Model(inputs, outputs, name="embedding_xlmroberta")


In [20]:
output = model.predict(lm_datasets['train']['input_ids'][0:1])

1/1 [==============================] - 2s 2s/step


In [21]:
output

array([[[ 0.22434333, -0.00958651, -0.18363872, ...,  0.16321908,
          0.07026199,  0.06239313],
        [ 0.46472907, -0.11707267, -0.25122482, ...,  0.14266491,
         -0.19166225,  0.14719823],
        [ 0.25792032,  0.11327289, -0.37167066, ...,  0.3510136 ,
         -0.10204633,  0.16656084],
        ...,
        [ 0.35631856, -0.07545323, -0.03211013, ...,  0.03959493,
         -0.25347534,  0.12344421],
        [ 0.11022241,  0.11060701, -0.33653867, ...,  0.14263879,
         -0.30762294,  0.19614282],
        [ 0.40933102,  0.04583286, -0.19882363, ...,  0.19835319,
         -0.3288786 ,  0.20452617]]], dtype=float32)